In [2]:
from manim import *
import networkx as nx
import numpy as np
import copy

Manim Community v0.16.0.post0

In [2]:
def imagenesArbol():
    matriz = [[SVGMobject("Food/plate.svg").scale(1.7), SVGMobject("Food/SalsaPomodoro.svg"), SVGMobject("Food/SalsaParmesano.svg"), 
               SVGMobject("Food/SalsaAjo.svg"), SVGMobject("Food/SalsaPesto.svg"), SVGMobject("Food/SalsaChampiñon.svg")],
               [SVGMobject("Food/espagueti.svg").scale(1.8).rotate(-1), SVGMobject("Food/Platillos/espagueti-pom.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-par.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-ajo.svg").scale(.7), 
                SVGMobject("Food/Platillos/espagueti-pesto.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-cham.svg").scale(.7)],
               [SVGMobject("Food/fusili.svg").rotate(-.1),SVGMobject("Food/Platillos/fusili-pom.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-par.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-ajo.svg").scale(.6), 
                SVGMobject("Food/Platillos/fusili-pesto.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-cham.svg").scale(.6)],
               [SVGMobject("Food/macaroni.svg").rotate(-.2),SVGMobject("Food/Platillos/macaroni-pom.svg").scale(.7),
                SVGMobject("Food/Platillos/macaroni-par.svg").scale(.7),
                SVGMobject("Food/Platillos/macaroni-ajo.svg").scale(.7), 
                SVGMobject("Food/Platillos/macaroni-pesto.svg").scale(.7),
                SVGMobject("Food/Platillos/macaroni-cham.svg").scale(.7)]]
    return matriz



In [3]:
def construirTabla(self, matriz, tipos1, tipos2):
    submatriz = [[matriz[i][j] for j in tipos2] for i in tipos1]

    first_column = [submatriz[i][0] for i in range(1, len(submatriz))]
    first_row = submatriz[0][1:]

    comb = [row[1:] for row in submatriz[1:]]
    
    table = MobjectTable(comb,
        row_labels=first_column,
        col_labels=first_row,
        include_outer_lines=True)
    
    return table


In [4]:
def distribuir_puntos_uniformemente(n, dist):
    puntos = []
    if n % 2 == 0:  # Si n es par, se divide niformemente a ambos lados de 0.
        cant_puntos_lado = n // 2
        puntos_lado = [dist * (i - 1/2) for i in range(1, cant_puntos_lado + 1)]
    else:  # Si n es impar, se coloca un punto en 0 y se divide el resto uniformemente.
        cant_puntos_lado = (n - 1) // 2
        puntos_lado = [dist * i for i in range(1, cant_puntos_lado + 1)]
        puntos += [0]
    
    # Distribuye puntos uniformemente arriba y abajo de 0 con una distancia de 0.5.
    
    puntos.extend(puntos_lado)
    puntos.extend(-1* np.array(puntos_lado))

    return sorted(puntos, reverse=True)

#submatriz = distribuir_puntos_uniformemente(6, .8)
#print(submatriz)

In [5]:
def distribucion_vertical(arreglo, combTotales, dist = .8):
    pisos = len(arreglo)
    distPisos = 2
    x = pisos * distPisos -6 # eje es x
    puntos_ultimo_renglon = distribuir_puntos_uniformemente(combTotales, dist)
    puntosAntes = puntos_ultimo_renglon
    p = [(x, valor, 0) for valor in puntosAntes]
    puntos = []
    puntos.append(p)
    aristas =[]
    for dec in reversed(arreglo):
        puntosR = []
        coor_ren = []
        x -= distPisos
        for i in range(0, len(puntosAntes), dec):
            valor_y = round((puntosAntes[i] + puntosAntes[i + dec - 1])/2,2)
            puntosR += [valor_y]
            coor_ren.append((x,valor_y, 0))
            for j in range(i , i+ dec):
                par = [(x,-1*valor_y, -5), (x + distPisos, -1*puntosAntes[j],-5)]
                aristas.insert(0,par)

        puntos.insert(0,coor_ren)
        puntosAntes = puntosR
    return (puntos, aristas)

#distribucion_vertical([2,3,1],6)

In [6]:
def distribucion_horizontal(arreglo, combTotales, dist = None):
    if dist is None:
        dist = 12 / combTotales
    pisos = len(arreglo)
    distPisos = 1.5 if pisos <= 4 else 1
    y = 3 - pisos * distPisos # eje es y
    puntos_ultimo_renglon = distribuir_puntos_uniformemente(combTotales, dist)
    puntosAntes = puntos_ultimo_renglon
    p = [(valor, y, 0) for valor in puntosAntes]
    puntos = []
    puntos.append(p)
    aristas = []
    for dec in reversed(arreglo):
        puntosR = []
        coor_ren = []
        y += distPisos
        for i in range(0, len(puntosAntes), dec):
            valor_x = round((puntosAntes[i] + puntosAntes[i + dec - 1])/2,2)
            puntosR += [valor_x]
            coor_ren.append((valor_x,y, 0))
            for j in range(i , i+ dec):
                par = [(-1*valor_x,y, -5), (-1*puntosAntes[j], y - distPisos,-5)]
                aristas.insert(0,par)

        puntos.insert(0,coor_ren)
        puntosAntes = puntosR

    return (puntos, aristas)


In [7]:
def multiplicar_valores(arreglo, dist = .8):
    if len(arreglo) == 0:
        return 0  # Si el arreglo está vacío, el resultado es 0.
    else:
        combTotales = np.prod(arreglo)
        if combTotales <= 10: # vertical
            # dist = .8
            puntos,aristas = distribucion_vertical(arreglo, combTotales, dist)

        else:
            puntos, aristas = distribucion_horizontal(arreglo, combTotales, dist)
        
        return puntos, aristas


In [8]:
def transpose_matrix(matrix):
    transpuesta = [[0] * len(matrix) for _ in range(len(matrix[0]))]

    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            transpuesta[j][i] = matrix[i][j]

    return transpuesta


In [9]:
def combPastaSencilla(matrizCompleta,tipos1, tipos2):
    submatriz = [[matrizCompleta[i][j] for j in tipos2] for i in tipos1]
    tipos2 = submatriz[0][1:]
    combinaciones = [fila[0] for fila in submatriz]
    
    for i in range(1, len(tipos1)):
        copias_tipos2 =[copy.deepcopy(combinacion) for combinacion in tipos2]
        combinaciones.extend(copias_tipos2)
    
    for j in range(1, len(tipos1)):
        combinaciones.extend(submatriz[j][1:])
            
    combinaciones
    
    return combinaciones

In [10]:
def construirArbol(matrizCompleta,tipos1,tipos2, dist = .8):
    
    dec1 = len(tipos1) - 1
    dec2 = len(tipos2) - 1
    valores = [dec1,dec2,1]
    coordenadas,aristas = multiplicar_valores(valores, dist)
    
    combinaciones = combPastaSencilla(matrizCompleta,tipos1,tipos2)
    arbolNiveles = []
    arbol = VGroup()
        
    i = 0
    j=0
    dec = [dec1,dec2,1]
    k = 0
    numAris = 1
    for piso in coordenadas:
        nivel = VGroup()
        for punto in piso:
            sc = .3
            if k == 1:
                sc = .4
            combinaciones[i].move_to(punto).scale(sc)
            arbol.add(combinaciones[i])
            nivel.add(combinaciones[i])
            i +=1 
        if k < len(dec):
            numAris *= dec[k]
            r = 0
            while j < len(aristas) and r < numAris:
                edge = Line(start=aristas[j][0], end=aristas[j][1])
                edge.z_index = -5
                #self.play(Write(edge))
                arbol.add(edge)
                nivel.add(edge)
                j += 1
                r += 1
            
        arbolNiveles.append(nivel)
        k += 1

    return (arbolNiveles, arbol, coordenadas,aristas)

In [11]:
def crearArbolNiveles(self,arbolNiveles):
    for obj in arbolNiveles:
        group = VGroup(*obj)
        self.add(group)
        self.play(FadeIn(group))

In [30]:
def imagenesArbolComplicado():
    matriz = [[[SVGMobject("Food/plate.svg").scale(1.7), SVGMobject("Food/SalsaPomodoro.svg"), 
               SVGMobject("Food/SalsaParmesano.svg"), SVGMobject("Food/SalsaAjo.svg"), 
               SVGMobject("Food/SalsaPesto.svg"), SVGMobject("Food/SalsaChampiñon.svg")],
               [SVGMobject("Food/espagueti.svg").scale(1.8).rotate(-1), SVGMobject("Food/Platillos/espagueti-pom.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-par.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-ajo.svg").scale(.7), 
                SVGMobject("Food/Platillos/espagueti-pesto.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-cham.svg").scale(.7)],
               [SVGMobject("Food/fusili.svg").rotate(-.1),SVGMobject("Food/Platillos/fusili-pom.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-par.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-ajo.svg").scale(.6), 
                SVGMobject("Food/Platillos/fusili-pesto.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-cham.svg").scale(.6)]],
             [[SVGMobject("Food/salmon.svg").scale(.7), SVGMobject("Food/SalsaPomodoro.svg"), 
              SVGMobject("Food/SalsaParmesano.svg"), SVGMobject("Food/SalsaAjo.svg"), 
              SVGMobject("Food/SalsaPesto.svg"), SVGMobject("Food/SalsaChampiñon.svg")],
               [SVGMobject("Food/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("Food/Platillos/espagueti-pom-salmon.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-par-salmon.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-ajo-salmon.svg").scale(.7), 
                SVGMobject("Food/Platillos/espagueti-pesto-salmon.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-cham-salmon.svg").scale(.7)],
               [SVGMobject("Food/fusili.svg").rotate(-.1),
                SVGMobject("Food/Platillos/fusili-pom-salmon.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-par-salmon.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-ajo-salmon.svg").scale(.6), 
                SVGMobject("Food/Platillos/fusili-pesto-salmon.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-cham-salmon.svg").scale(.6)]],
             [[SVGMobject("Food/carne.svg").scale(.7), SVGMobject("Food/SalsaPomodoro.svg"), 
              SVGMobject("Food/SalsaParmesano.svg"), SVGMobject("Food/SalsaAjo.svg"), 
              SVGMobject("Food/SalsaPesto.svg"), SVGMobject("Food/SalsaChampiñon.svg")],
               [SVGMobject("Food/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("Food/Platillos/espagueti-pom-carne.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-par-carne.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-ajo-carne.svg").scale(.7), 
                SVGMobject("Food/Platillos/espagueti-pesto-carne.svg").scale(.7),
                SVGMobject("Food/Platillos/espagueti-cham-carne.svg").scale(.7)],
               [SVGMobject("Food/fusili.svg").rotate(-.1),
                SVGMobject("Food/Platillos/fusili-pom-carne.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-par-carne.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-ajo-carne.svg").scale(.6), 
                SVGMobject("Food/Platillos/fusili-pesto-carne.svg").scale(.6),
                SVGMobject("Food/Platillos/fusili-cham-carne.svg").scale(.6)]]]
    return matriz


In [7]:
def combPasta(matrizCompleta,tipos1, tipos2, tipos3):
    submatriz = [[[matrizCompleta[i][j][k] for k in tipos2] for j in tipos1] for i in tipos3]
    n = len(tipos3) -1
    p = len(tipos2) -1
    k = len(tipos1) -1
    tipos2 = submatriz[0][0][1:]

    tipos1 = [submatriz[0][i][0] for i in range(1, len(submatriz[0]))]

    tipos3 = [submatriz[i][0][0] for i in range(1, len(submatriz))]


    combinaciones = [matrizCompleta[0][0][0]]
    combinaciones.extend(tipos1)

    for i in range(0, len(tipos1)):
        copias_tipos2 =copy.deepcopy(tipos2)
        combinaciones.extend(copias_tipos2)

    for i in range(0, len(tipos1)*len(tipos2)):
        copias_tipos3 =copy.deepcopy(tipos3)
        combinaciones.extend(copias_tipos3)

    for i in range(0, k):
        for r in range(0, p):
            for j in range(0,n):
                copia_tipos3 = copy.deepcopy(submatriz[j+1][(i+1)][r+1])
                combinaciones.append(copia_tipos3)
    return combinaciones

In [8]:
def construirArbolCompl(matrizCompleta,tipos1,tipos2, tipos3,dist = .8):
    
    dec1 = len(tipos1) - 1
    dec2 = len(tipos2) - 1
    dec3 = len(tipos3) - 1
    valores = [dec1,dec2,dec3,1]
    coordenadas,aristas = multiplicar_valores(valores, dist)
    
    combinaciones = combPasta(matrizCompleta,tipos1,tipos2, tipos3)
    arbolNiveles = []
    arbol = VGroup()
        
    i = 0
    j=0
    dec = [dec1,dec2,dec3,1]
    k = 0
    numAris = 1
    for piso in coordenadas:
        nivel = VGroup()
        for punto in piso:
            sc = .25
            if k == 1:
                sc = .3
            combinaciones[i].move_to(punto).scale(sc)
            arbol.add(combinaciones[i])
            nivel.add(combinaciones[i])
            i +=1 
        if k < len(dec):
            numAris *= dec[k]
            r = 0
            while j < len(aristas) and r < numAris:
                edge = Line(start=aristas[j][0], end=aristas[j][1])
                edge.z_index = -5
                #self.play(Write(edge))
                arbol.add(edge)
                nivel.add(edge)
                j += 1
                r += 1
            
        arbolNiveles.append(nivel)
        k += 1

    return (arbolNiveles, arbol, coordenadas,aristas)